# Introduction
Before we get started with this, it's suggested that you skim through the [startup guide](https://github.com/TheProjectsGuy/Blogger/blob/master/OpenCV/OpenCV_Tutorials/Introduction/Sample-OpenCV.ipynb) I've made so that we're all on the same platform.

# Video Handling using OpenCV
Say you have a video file on disk, which is basically a series of images stacked together in a single file, and you want to read it. Let's see how we can read this file

Let's start with importing the library

In [7]:
# Import the cv2 library
import cv2 as cv

## Reading frames in a video
The same function that is used to get webcam feed also helps us get frames from a video on disk. It's the `cv.VideoCapture` function, but now we don't pass it a number (because this isn't a camera we're dealing with). We pass the function the full path with name and extension of the video file that we want to read. We can read video frame by frame and display it.
### Frame rate
We need to keep one thing in mind while viewing recorded video. As you might have seen that the timeout delay that we passed to the `cv.waitKey` function in the previous tutorial is 1 millisecond, but if we do the same thing here, it will read new frames every one millisecond. The function is basically the time delay we cause between processing two consecutive frames in a video. The entire video will get over in a very short period of time if this is too low. The video will appear to be in slo-mo if this delay is high. So we can adjust this timeout period to cause slo-mo or time-lapse. Let's see how you can do that.

- [cv.VideoCapture](https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_video_display/py_video_display.html#playing-video-from-file): Strem video from a source

In [8]:
# Video file name (must be an existing file)
file = "Keyboard - 10822.mp4"
# Delay that we do in between reading two frames (in milliseconds)
tdelay = 20
# Video reading object
vid_reader = cv.VideoCapture(file)

Now let's see how we can read the file using the `cv.VideoCapture.read` function. This function returns two things, if the reading object found an image to return and the image (if it found something to return). If the file doesn't exist or when we have finished reading the file and continue to ask for images, it returns **None** for the image.

- [cv.VideoCapture.read](https://docs.opencv.org/3.4/d8/dfe/classcv_1_1VideoCapture.html#a473055e77dd7faa4d26d686226b292c1): Read a frame (image) using a video reader object.
- [cv.destroyAllWindows](https://docs.opencv.org/3.4/d7/dfc/group__highgui.html#ga6b7fc1c1a8960438156912027b38f481): Used to close all windows that were created by opencv before the calling of this function
- [cv.VideoCapture.release](https://docs.opencv.org/3.4/d8/dfe/classcv_1_1VideoCapture.html#afb4ab689e553ba2c8f0fec41b9344ae6): Used to free up the resources that the video reader is taking. This must be called when you're done with reading the file.

In [9]:
# Keep reading the file till end
while (vid_reader.isOpened()):
    # Use the read function
    ret, img = vid_reader.read()
    # If there's nothing to read, then break out of the loop
    if not ret:
        break
    # View the image in a window named "Frame"
    cv.imshow('Frame', img)
    # Wait for a timeout and continue execution
    key = cv.waitKey(tdelay) & 0xFF
    # If user presses the 'q' key, break out of the loop
    if key == ord('q'):
        break
# Destroy the frame window after all the work is done
cv.destroyAllWindows()
# Release the resources that the video reader is taking
vid_reader.release()

You can try the following things on your own
<br><input type="checkbox"> Try using different timeouts given to *cv.waitKey* function. Namely, experiment with different values of `tdelay`.
<br><input type="checkbox"> Try reading a different video file.

# Video Recording using OpenCV
Let's build a video recorder using OpenCV. We will be able to adjust a good number of properties for it, so that it's customizable. 

There are a few properties that we must decide upon before moving ahead, so let's see them.

## Properties to be adjusted
Every recorded video has some basic features like filename, FPS, size of frames and if it's in color or grayscale. We additionally have some more properties which you'll see below. The camera number is the index of the camera to be used (starting from 0).

| Variable | Description |
| ---- | ---- |
| cam_no | Camera number |
| dest_file_name | Name of the file to be saved (complete path with extension) |
| fps | The FPS (frames per second) of the video file |
| frame_size | The size (resolution) of the frames to be saved (width by height)<br> Make sure that it's of the same size as you're getting from the webcam|

Let's make variables for all of them.

In [ ]:
# Variables for the program
# Camera number
cam_no = 1
# File name (complete path with extension) where the video is to be stored
dest_file_name = 'RecordedFile.avi'
# FPS of our video file
fps = 20.0
# The size of our frame
frame_size = (640, 480)

## Recording and Video stream objects
Now, we declare a few variables that'll help us get frames from the camera and help us write some video on the disk.
- The *first* object is for the *video camera*, pretty straight forward
- The *secong* object is for the video recorder to know some basic details about the format. It's called a **Four Character Code**, [fourcc](https://en.wikipedia.org/wiki/FourCC) in short. The various options are available [here](http://www.fourcc.org/codecs.php). Be careful about this choice, not all operating systems support all of them. A common one is XVID code (for MPEG-4) which is widely supported. We would normally pass it character by character, that is ('X', 'V', 'I', 'D'), but if we use the **\*** operator in python, we can use (\*'XVID') and it'll do the same thing.
- The *third* object is the object that will *write the video* on disk. It requires the name of file, the foucc object, fps and frame_size. Keep in mind, every output stream requires a unique recorder object but they can share the fourcc object. That is, if you want to save to multiple places, you will need multiple video writing objects but only one fourcc object.

You'll learn about the following here
- [cv.VideoCaptute](https://docs.opencv.org/3.0-beta/modules/videoio/doc/reading_and_writing_video.html#videocapture): Used to get frames from a webcam or a video.
- [cv.VideoWriter_fourcc](https://docs.opencv.org/3.0-beta/modules/videoio/doc/reading_and_writing_video.html#videowriter-fourcc): Used to declare the FCC for a video recording.
- [cv.VideoWriter](https://docs.opencv.org/3.0-beta/modules/videoio/doc/reading_and_writing_video.html#videowriter): Used to write video on disk, by adding images frame by frame.

In [ ]:
# Objects declared
# Camera object
cam = cv.VideoCapture(cam_no)
# FourCC object
rec_fourcc = cv.VideoWriter_fourcc(*'XVID')
# Recorder object
rec = cv.VideoWriter(dest_file_name, rec_fourcc, fps, frame_size)

## Record 
Now, let's write the code to record feed from the webcam specified earlier.

In a loop
- We'll get the frames from camera
- Show them for preview
- Write them to the file (and also keep a tally of the number of frames written)
- If we get a `q` from keyboard, we break from the loop

Keep in mind that the value passed to the `cv.waitKey` function is 1 and not 0, so that it resumes operations after a one millisecond timeout. This does **not** affect our FPS since the *VideoWriter* object takes care of the file that is saved has the specified FPS, so you needn't worry about the exact value.

- [cv.VideoWriter.write](https://docs.opencv.org/3.0-beta/modules/videoio/doc/reading_and_writing_video.html#videowriter-write): Used to append an image frame in a video using a video writer.

In [ ]:
# Recording loop

# To keep track of the frame number
fno = 0
# The loop
while cam.isOpened():
    # Get a frame from the camera
    ret, frame = cam.read()
    if not ret:   # If resource is not available or if we don't get frames, break out
        break
    # Preview frames
    cv.imshow("Live Feed", frame)
    # Save the frame on disk in the video file
    rec.write(frame)
    fno += 1
    # Keyboard entry
    key = cv.waitKey(1) & 0xff
    if key == ord('q'):
        break

## Release resources and cleanup
As you already know, all these streams must be released. The *VideoWriter* object needs to be released so that we put a proper termination to the video file (else it may get corrupted). We also destroy any windows opened by OpenCV. We end by printing the number of frames we recorded.

In [ ]:
# Release camera resources
cam.release()
# Release video resources
rec.release()
# Destroy all windows created
cv.destroyAllWindows()
print("{num_frames} frames written to file \"{fname}\"".format(num_frames=fno, fname=dest_file_name))

## Try it on your own
Try the following things on your own
<br><input type="checkbox"> Try giving an invalid directory as file name and see what happens
<br><input type="checkbox"> Experiment with different timeouts for `cv.waitKey` and different FPS
<br><input type="checkbox"> Connect a webcam and use some other index, try and see what happens when you give an index out of the available ranges.
<br><input type="checkbox"> Try converting images to different formats (like HSV, grayscale, ...) before saving them and then view the results
<br><input type="checkbox"> See what happens when you have a frame_size something other than the size of the frames you're writing to the file (Hint : OpenCV has a function to resize images, `cv.resize`. More information [here](https://docs.opencv.org/3.4.2/da/d6e/tutorial_py_geometric_transformations.html))
<br><input type="checkbox"> Apply some effects to the frames before writing them to the file. More information about this [here](https://docs.opencv.org/3.4.2/d3/df2/tutorial_py_basic_ops.html)